# Julia 型別 (Types) 系統簡介

## 1. 型別 (Type) 的宣告

型別的宣告是使用 `::` 運算子  可以針對變數、表達式 (expression)、與函式進行型別的宣告.

### 變數 (variable)

宣告變數型別, 只能針對 local 變數, 若宣告 global 變數的型別時會產生錯誤.

In [1]:
local x::Int8 = 10

10

In [2]:
# global 變數
y::Int64 = 10

ErrorException: syntax: type declarations on global variables are not yet supported

In [3]:
function foo()
    x::Int32 = 100
    x
end

foo()

100

呼叫 `foo()` 後, 我們可以看到回傳值的型別是 `Int32`

In [4]:
# ans 是 Julia 語言保留字, 代表的是目前最後計算值
typeof(ans)

Int32

### 表達式 (expression)

In [5]:
(1+2)::Int64

3

若是 Type assertion 傳回 false 的話, 會產生 exception.

In [6]:
(1+2)::Float64

TypeError: TypeError: in typeassert, expected Float64, got Int64

### 函式 (function)

若宣告函式回傳型別的話, 回傳值會進行轉型為指定的型別, 若無法轉型的話則會產生錯誤.

In [7]:
function foo()::Float64
    return 2 / 2
end

foo()

1.0

In [8]:
typeof(ans)

Float64

## 2. 常用函式與型別階層

下圖為 `Number` 及其子型別，`Any` 為所有型別的父型別。

![](./Julia_Number.png)

呼叫 `supertype()` 函式查看某一型別的父型別.

In [9]:
supertype(Real)

Number

呼叫 `subtypes()` 函式查看某一型別的子型別.

In [10]:
subtypes(Number)

2-element Array{Any,1}:
 Complex
 Real

型別的父、子關係，是使用 `<:` 與 `>:` 運算子來指定。

查看型別之間是否為父子關係。

In [11]:
<:(Integer, Number)

true

In [12]:
# 也可以直接使用運算子
Integer <: AbstractFloat

false

In [13]:
# Integer 也是 Int64 的父型別
Integer >: Int64

true

## 3. 抽象型別 (Abstract Type)

抽象型別有幾個特性：
- 無法被實例化 (Instantiate)
- 抽象是可以延伸的, 可以有多層次
- 可以運用抽象型別來撰寫泛用 (generic) 函式，以做為函式的基本行為 (behavior)

抽象型別的最上層是 `Any`, 也是所有型別的最上層父型別; 抽象型別的最下層則是 `Union{}`, 也是所有型別的子型別.

以 `Number` 型別為例, 可以看到其為所有數值型別的父型別, 而且是抽象型別.

In [14]:
? Number

search: Number LineNumberNode VersionNumber



```
Number
```

Abstract supertype for all number types.


### 抽象型別的宣告

`abstract type <<name>> end`

宣告時可使用 `<:` 運算子指定其父型別. 如果沒有指定的話, 則父型別為 `Any`.

`abstract type <<name>> <: <<supertype>> end`

In [15]:
abstract type YetAnotherNumber <: Number end

下列範例是當函式沒有指定型別時其型別為抽象型別 `Any`, 傳入整數引數 (argument) 值時, Julia 會內部定義並編譯符合整數引數的 Method 並執行, 所以仍然可以正確地計算出整數之回傳值.

In [16]:
function xyplus(x, y)
    x + y
end

xyplus (generic function with 1 method)

In [17]:
xyplus(2, 5)

7

## 4. 原始型別 (Primitive Type)

### 原始型別的宣告

`primitive type <<name>> <<bits>> end`

宣告時可使用 `<:` 運算子指定其父型別. 如果沒有指定的話, 則父型別為 `Any`.

`primitive type <<name>> <: <<supertype>> <<bits>> end`

bits 數字是指定型別需要的儲存空間.

範例: Int64 的宣告

`primitive type Int64 <: Signed 64 end`

In [19]:
primitive type AnotherNumericType <: Signed 64 end

## 5. 複合型別 (Composite Type)

複合型別是用關鍵字 `struct` 來進行宣告，成員可以指定型別或不指定，不指定即為 `Any` 型別；成員的型別也可以是另一個複合型別。

`struct` 可分為可變 (mutable) 與不可變的，其差異在於在實例化 (instantiate) 之後 mutable struct 的成員值是可以被改變的，而不可變複合型別成員值不可以被改變，嘗試改變時會產生錯誤。

下例為宣告宣告不可變的複合型別。

In [32]:
struct Bar
    m::Int64
    n::Float64
end

In [33]:
struct Foo
    aa::Bar
    bb::Int
    cc::Float64
end

In [36]:
b1 = Foo(Bar(1, 2.0), 3, 4.0)

Foo(Bar(1, 2.0), 3, 4.0)

In [37]:
b1.bb = 2

ErrorException: setfield! immutable struct of type Foo cannot be changed

In [38]:
b2.aa.n

2.0

呼叫 `dump()` 函式可以查看複合型別的內部結構。

In [22]:
dump(Foo)

Foo <: Any
  aa::Bar
  bb::Int64
  cc::Float64


在複合型別內的複合型別成員透過下列語法存取。

In [23]:
fieldnames(Foo)

(:aa, :bb, :cc)

可變的複合型別，在宣告時使用 `mutable` 保留字。

In [24]:
mutable struct Foo2
    aa::Bar
    bb::Int
    cc::Float64
end

In [25]:
b2 = Foo2(Bar(1, 2.0), 3, 4.0)

Foo2(Bar(1, 2.0), 3, 4.0)

In [26]:
b2.bb = 2

2

複合型別的成員是可以被存取的，例如：

In [27]:
b2.aa

Bar(1, 2.0)

在複合型別內的複合型別成員透過下列語法存取。

In [28]:
b2.aa.n

2.0

## 6. 型別聯合 (Union)

型別聯合是透過 Union 的方式，可以讓物件的型別限定在數個指定的型別之內，但又不需要使用更高層級或是抽象型別來宣告，例如：下列語法示範了，如果我們希望型別可能是 Int64、Int32、或是 Float64 時，可以透過 Union 來達到。

In [39]:
Int64OrInt32OrFloat64 = Union{Int64, Float64, Int32}

Union{Float64, Int32, Int64}

In [40]:
Int64OrInt32OrFloat64(1)

1

如果型別不在 Union 中，則會出現 `TypeError`。

In [41]:
a = Int8(1)
a::Int64OrInt32OrFloat64

TypeError: TypeError: in typeassert, expected Union{Float64, Int32, Int64}, got Int8

In [42]:
typeof(Int64OrInt32OrFloat64)

Union

可以看到 Union 的型別是 `DataType`。

In [43]:
typeof(Union)

DataType

其父型別為 `Type{T}`，是參數化型別。

In [45]:
supertype(Union)

Type{T}

如果 `Union` 內參數為空，即 `Union{}`，則其為特殊型別，是最底層的型別，是所有型別的子型別。

In [46]:
typeof(Union{})

Core.TypeofBottom

In [47]:
subtypes(Union{})

0-element Array{Type,1}

## 7. 型別參數化 (Parametric Types)

型別參數化可以彈性地型別，而不影響程式的彈性及重覆使用性，也能降低程式的複雜度。型別參數化分為兩個部分介紹：
- 複合型別參數化
- 抽象型別參數化

### 複合型別參數化

複合型別參數化的宣告語法：`struct <<name>>{T}`

In [48]:
struct Point{T}
    x::T
end

### 抽象型別參數化

下例是抽象型別 `Pointy` 參數化，而 `Pointy` 有三個子型別，分別代表一維、二維、三維座標型別而有不同的成員。型別階層如下圖：

![](./pointy.png)

抽象化型別參數化的宣告語法：`abstract type <<name>>:{T} end`

In [49]:
abstract type Pointy{T} end

In [50]:
struct Point1D{T} <: Pointy{T}
    x::T
end

In [51]:
struct Point2D{T} <: Pointy{T}
    x::T
    y::T
end

In [52]:
struct Point3D{T} <: Pointy{T}
    x::T
    y::T
    z::T
end

函式之引數宣告為抽象型別，在函式內判斷傳入的複合型別為何，並印出傳入的成員值。

透過 `isa()` 函式判斷型別，若型別相同則為 `true`，若不同則為 `false`。

In [53]:
function print_members(p::Pointy)
    if isa(p, Point1D)
        println(p.x)
    elseif isa(p, Point2D)
        println(p.x, " ", p.y)
    else
        println(p.x, " ", p.y, " ", p.z)
    end
end

print_members (generic function with 1 method)

In [54]:
p2 = Point2D(1.0, 2.0)

Point2D{Float64}(1.0, 2.0)

In [55]:
print_members(p2)

1.0 2.0
